In [297]:
import pandas as pd
import re
from glob import glob
import numpy as np
from pulp import *

In [3]:
allleagues1517 = pd.read_csv('/Users/jbachlombardo/Documents/Python/Projects/xG/AllLeagues_15-17.csv')

In [23]:
fnames1718 = glob('/Users/jbachlombardo/Documents/Python/Projects/xG/*1718.csv')
fnames1819 = glob('/Users/jbachlombardo/Documents/Python/Projects/xG/*1819.csv')

In [17]:
seasons1718 = pd.DataFrame()
for f in fnames1718 :
    _ = pd.read_csv(f)
    _['Season'] = '2017/18'
    if f[50:-8] == 'bundes' :
        _['League'] = 'Bundesliga'
    elif f[50:-8] == 'liga' :
        _['League'] = 'La Liga'
    elif f[50:-8] == 'ligue1' :
        _['League'] = 'Ligue 1'
    elif f[50:-8] == 'seriea' :
        _['League'] = 'Serie A'
    else :
        _['League'] = 'EPL'
    seasons1718 = pd.concat([seasons1718, _])

In [24]:
seasons1819 = pd.DataFrame()
for f in fnames1819 :
    _ = pd.read_csv(f)
    _['Season'] = '2018/19'
    if f[50:-8] == 'bundes' :
        _['League'] = 'Bundesliga'
    elif f[50:-8] == 'liga' :
        _['League'] = 'La Liga'
    elif f[50:-8] == 'ligue1' :
        _['League'] = 'Ligue 1'
    elif f[50:-8] == 'seriea' :
        _['League'] = 'Serie A'
    else :
        _['League'] = 'EPL'
    seasons1819 = pd.concat([seasons1819, _])

In [29]:
allleagues1517 = allleagues1517.rename(columns = {'Goals': 'G', 'Assists': 'A'})

In [33]:
cols = allleagues1517.columns
cols

Index(['Season', 'League', 'Team', 'Player', 'Pos', 'Apps', 'Min', 'G', 'A',
       'Sh90', 'KP90', 'xG', 'xA', 'xG90', 'xA90'],
      dtype='object')

In [31]:
seasons1819 = seasons1819.rename(columns = {'team': 'Team'})
seasons1718 = seasons1718.rename(columns = {'team': 'Team'})

In [34]:
seasons1819 = seasons1819[cols]
seasons1718 = seasons1718[cols]

In [36]:
allseasons = pd.concat([allleagues1517, seasons1718, seasons1819])

In [38]:
allseasons = allseasons.dropna()

In [40]:
allseasons['xG'] = allseasons['xG'].apply(lambda x: float(re.split('[+-]', x)[0]))
allseasons['xA'] = allseasons['xA'].apply(lambda x: float(re.split('[+-]', x)[0]))

In [43]:
allseasons['xG+/-'] = allseasons['xG'] - allseasons['G']
allseasons['xA+/-'] = allseasons['xA'] - allseasons['A']

In [46]:
allseasons.to_csv('/Users/jbachlombardo/Documents/Python/Projects/xG/all_seasons_leagues_1415-1819.csv')

In [56]:
costs = pd.read_csv('/Users/jbachlombardo/Documents/Python/Projects/xG/fpl_costs_120819.csv')

In [61]:
costs = costs.dropna(axis = 1)

In [68]:
team_dict = {
'ARS': 'Arsenal',
'AVL': 'Aston Villa',
'BHA': 'Brighton Hove Albion',
'BOU': 'Bournemouth',
'BUR': 'Burnley',
'CHE': 'Chelsea',
'CRY': 'Crystal Palace',
'EVE': 'Everton',
'LEI': 'Leicester',
'LIV': 'Liverpool',
'MCI': 'Manchester City',
'MUN': 'Manchester United',
'NEW': 'Newcastle United',
'NOR': 'Norwich',
'SHU': 'Sheffield United',
'SOU': 'Southampton',
'TOT': 'Tottenham',
'WAT': 'Watford',
'WHU': 'West Ham',
'WOL': 'Wolverhampton Wanderers'
}

In [69]:
costs['Team_name'] = costs['Team'].map(team_dict)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [77]:
costs = costs.rename(columns = {'Name': 'Player'})

In [254]:
def get_position_cost(x, y) :
    last = x.split()[-1]
    results = costs.loc[(costs['Player'].str.contains(last)) & (costs['Team_name'] == y), ['Cost', 'Position']].values
    try :
        return list(results[0])
    except :
        return [np.nan, np.nan]

In [259]:
allseasons_results = allseasons[['Player', 'Team']].apply(lambda x: get_position_cost(x[0], x[1]), axis = 1)

In [260]:
allseasons[['FPL_Cost', 'FPL_Pos']] = pd.DataFrame(allseasons_results.values.tolist(), index = allseasons_results.index)

In [291]:
have_costs = np.sort(allseasons.dropna(subset = ['FPL_Cost'])['Player'].unique())

In [292]:
all_players_costs = costs['Player'].unique()

In [293]:
allseasons.to_csv('/Users/jbachlombardo/Documents/Python/Projects/xG/all_seasons_leagues_1415-1819_withcosts.csv')

In [295]:
with_costs = allseasons.dropna(subset = ['FPL_Cost'])

In [304]:
with_costs['Sh'] = with_costs['Sh90'] * with_costs['Min'] / 90
with_costs['KP'] = with_costs['KP90'] * with_costs['Min'] / 90

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [451]:
agg_dict = {
    'xG': 'sum',
    'xA': 'sum',
    'G': 'sum',
    'A': 'sum',
    'Min': 'sum',
    'Sh': 'sum',
    'KP': 'sum',
    'Season': 'count',
    'FPL_Cost': 'max'
}
players_totals = with_costs.groupby(['Player', 'Team', 'FPL_Pos'], as_index = False).agg(agg_dict)

In [452]:
rate_cols = ['xG', 'xA', 'G', 'A', 'Sh', 'KP']
for r in rate_cols :
    new_col = r + '_90'
    players_totals[new_col] = players_totals[r] / players_totals['Min'] * 90

In [453]:
players_totals['xG+/-'] = players_totals['xG'] - players_totals['G']
players_totals['xA+/-'] = players_totals['xA'] - players_totals['A']

In [454]:
players_totals['xG+/-_90'] = players_totals['xG_90'] - players_totals['G_90']
players_totals['xA+/-_90'] = players_totals['xA_90'] - players_totals['A_90']

In [455]:
players_three = with_costs[(with_costs['Season'] == '2016/17') | (with_costs['Season'] == '2017/18') | (with_costs['Season'] == '2018/19')]

In [456]:
players_two = with_costs[(with_costs['Season'] == '2017/18') | (with_costs['Season'] == '2018/19')]

In [457]:
players_one = with_costs[with_costs['Season'] == '2018/19']

In [458]:
players_three = players_three.groupby(['Player', 'Team', 'FPL_Pos'], as_index = False).agg(agg_dict)

In [459]:
for r in rate_cols :
    new_col = r + '_90'
    players_three[new_col] = players_three[r] / players_three['Min'] * 90

In [460]:
players_three['xG+/-'] = players_three['xG'] - players_three['G']
players_three['xA+/-'] = players_three['xA'] - players_three['A']
players_three['xG+/-_90'] = players_three['xG_90'] - players_three['G_90']
players_three['xA+/-_90'] = players_three['xA_90'] - players_three['A_90']

In [461]:
players_two = players_two.groupby(['Player', 'Team', 'FPL_Pos'], as_index = False).agg(agg_dict)

In [462]:
for r in rate_cols :
    new_col = r + '_90'
    players_two[new_col] = players_two[r] / players_two['Min'] * 90

In [463]:
players_two['xG+/-'] = players_two['xG'] - players_two['G']
players_two['xA+/-'] = players_two['xA'] - players_two['A']
players_two['xG+/-_90'] = players_two['xG_90'] - players_two['G_90']
players_two['xA+/-_90'] = players_two['xA_90'] - players_two['A_90']

In [464]:
for r in rate_cols :
    new_col = r + '_90'
    players_one[new_col] = players_one[r] / players_one['Min'] * 90

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [465]:
players_one['xG+/-'] = players_one['xG'] - players_one['G']
players_one['xA+/-'] = players_one['xA'] - players_one['A']
players_one['xG+/-_90'] = players_one['xG_90'] - players_one['G_90']
players_one['xA+/-_90'] = players_one['xA_90'] - players_one['A_90']

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pand

`score = 2*(2*xG_total + xG_90) + 2*xA_total + xA_90 /// 5 (0.1), 3 (0.35), 1 (0.55) year intervals`

In [466]:
players_totals['Score_5'] = 4*players_totals['xG'] + 2*players_totals['xG_90'] + 2*players_totals['xA'] + players_totals['xA_90']

In [467]:
players_three['Score_3'] = 4*players_three['xG'] + 2*players_three['xG_90'] + 2*players_three['xA'] + players_three['xA_90']
players_two['Score_2'] = 4*players_two['xG'] + 2*players_two['xG_90'] + 2*players_two['xA'] + players_two['xA_90']
players_one['Score_1'] = 4*players_one['xG'] + 2*players_one['xG_90'] + 2*players_one['xA'] + players_one['xA_90']

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [468]:
index_cols = ['Player', 'Team', 'FPL_Pos', 'FPL_Cost']

In [469]:
players_totals = players_totals.set_index(index_cols)
players_three = players_three.set_index(index_cols)
players_two = players_two.set_index(index_cols)
players_one = players_one.set_index(index_cols)

In [470]:
player_scores = pd.DataFrame(index = players_totals.index)

In [471]:
player_scores['Score_5'] = players_totals['Score_5'] * 0.05
player_scores['Score_3'] = players_three['Score_3'] * 0.15
player_scores['Score_2'] = players_two['Score_2'] * 0.30
player_scores['Score_1'] = players_one['Score_1'] * 0.50

In [472]:
player_scores['Score_Total'] = player_scores.sum(axis = 1)

In [474]:
player_scores = player_scores.reset_index()

In [476]:
players = player_scores['Player'].values
fwds = player_scores[player_scores['FPL_Pos'] == 'FWD']['Player'].values
mids = player_scores[player_scores['FPL_Pos'] == 'MID']['Player'].values
defs = player_scores[player_scores['FPL_Pos'] == 'DEF']['Player'].values

In [484]:
player_points = player_scores.set_index('Player')['Score_Total'].to_dict()
player_costs = player_scores.set_index('Player')['FPL_Cost'].to_dict()
player_teams = player_scores.set_index('Player')['Team'].to_dict()

In [480]:
fantasy = LpProblem('max_score_fantasy', LpMaximize)

In [481]:
player_vars = LpVariable.dicts('Player_', players, cat = 'Binary')

In [482]:
fantasy += lpSum([player_points[i] * player_vars[i] for i in players]), 'max_points'

In [485]:
fantasy += lpSum([player_costs[p]*player_vars[p] for p in players]) <= 90, 'spending_limit'
fantasy += lpSum([player_vars[p] for p in players if p in fwds]) == 3, 'forwards'
fantasy += lpSum([player_vars[p] for p in players if p in mids]) == 5, 'midfielders'
fantasy += lpSum([player_vars[p] for p in players if p in defs]) == 5, 'defenders'

In [486]:
fantasy.solve()

1

In [487]:
LpStatus[fantasy.status]

'Optimal'

In [490]:
in_team = {}
for v in fantasy.variables() :
    name = v.name[8:].replace('_', ' ')
    in_team[name] = v.varValue

In [491]:
player_scores['In_team'] = player_scores['Player'].map(in_team)

In [495]:
player_scores[player_scores['In_team'] == 1]

,Player,Team,FPL_Pos,FPL_Cost,Score_5,Score_3,Score_2,Score_1,Score_Total,In_team
33,Ashley Barnes,Burnley,FWD,6.5,7.392847,18.234116,27.135373,30.730492,83.492828,1.0
85,David Silva,Manchester City,MID,8.0,11.321802,22.415202,32.505732,26.859138,93.101875,1.0
158,Jamie Vardy,Leicester,FWD,9.0,17.410824,30.795127,46.878452,44.092445,139.176848,1.0
188,Joshua King,Bournemouth,FWD,6.5,8.895418,22.201482,29.866810,31.832447,92.796157,1.0
219,Luka Milivojevic,Crystal Palace,MID,7.0,4.625749,13.877246,24.993660,24.690658,68.187313,1.0
243,Matt Ritchie,Newcastle United,DEF,5.5,2.716964,8.150893,16.301785,14.202327,41.371969,1.0
251,Maya Yoshida,Southampton,DEF,4.5,1.923336,4.003686,5.626014,4.870033,16.423068,1.0
260,Mohamed Salah,Liverpool,MID,12.5,11.348409,34.045227,68.090454,54.792899,168.276988,1.0
268,Nathan Aké,Bournemouth,DEF,5.0,2.170953,6.512860,10.116832,10.557346,29.357991,1.0
279,Patrick van Aanholt,Crystal Palace,DEF,5.5,2.202132,6.606395,11.846154,10.485315,31.139996,1.0
